# Тетрадка с моделью классификации

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

Mounted at /content/drive


In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 27.5 MB/s 
     |████████████████████████████████| 163 kB 61.1 MB/s 
     |████████████████████████████████| 6.6 MB 70.7 MB/s 


In [ ]:
! pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 30.1 MB/s 
     |████████████████████████████████| 81 kB 12.2 MB/s 
     |████████████████████████████████| 209 kB 70.2 MB/s 
     |████████████████████████████████| 78 kB 10.2 MB/s 
     |████████████████████████████████| 147 kB 60.0 MB/s 
     |████████████████████████████████| 112 kB 70.5 MB/s 
     |████████████████████████████████| 49 kB 8.3 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=828b5aa6116ba7da8954cde5556b4c844745a658a22665490be70b8fca58dcbe
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
! pip install sentencepiece==0.1.91

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 29.0 MB/s 


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AdamW, get_linear_schedule_with_warmup
import time
import datetime
import random
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import re
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from optuna.pruners import SuccessiveHalvingPruner
from optuna.samplers import TPESampler

In [ ]:
torch.cuda.amp.autocast(enabled=True)

In [ ]:
# set random seed to get reproducible results

SEED = 15
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
torch.backends.cudnn.deterministic = True

# tell pytorch to use cuda
device = torch.device("cuda")

Get data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/kursovaya2022/1000_entries.csv')

In [ ]:
df

,Unnamed: 0,text,clean_text,direct_speech,clean_direct_speech,direct_speech_wo_author,author_remark,clean_author_remark,verb,masked_text,maske_ds,verb_lemma
0,0,"Разве мог Александр Мелявский, ступая на перро...","Разве мог Александр Мелявский, ступая на перро...","<said aloud='True' type='direct'>-Ребята, дава...","-Ребята, давайте сходим в воскресенье на стади...","-Ребята, давайте сходим в воскресенье на стадион","<speech_verb semantic=""speech"" emotion=""neutra...","сказала Таня, когда парни подошли к девочкам, ...",сказала,"Разве мог Александр Мелявский, ступая на перро...","-Ребята, давайте сходим в воскресенье на стади...",сказать
1,2,"Разве мог Александр Мелявский, ступая на перро...","Разве мог Александр Мелявский, ступая на перро...",<said aloud='True' type='direct'>- Он здесь</s...,"- Он здесь! - Сказал Александр, подойдя к Миха...",- Он здесь,"<speech_verb semantic=""speech"" emotion=""neutra...","Сказал Александр, подойдя к Михал Иванычу.",сказал,"Разве мог Александр Мелявский, ступая на перро...","- Он здесь! - [MASK] Александр, подойдя к Миха...",сказать
2,3,"Разве мог Александр Мелявский, ступая на перро...","Разве мог Александр Мелявский, ступая на перро...",<said aloud='True' type='direct'>- Катенька ид...,"- Катенька иди, мы сами справимся. - Сказала о...","- Катенька иди, мы сами справимся","<speech_verb semantic=""speech"" emotion=""neutra...","Сказала одна из женщин, избавив парня, от нело...",сказала,"Разве мог Александр Мелявский, ступая на перро...","- Катенька иди, мы сами справимся. - [MASK] од...",сказать
3,4,"Разве мог Александр Мелявский, ступая на перро...","Разве мог Александр Мелявский, ступая на перро...",<said aloud='True' type='direct'>- Катенька ид...,"- Катенька иди, мы сами справимся. - Сказала о...",. -,", -",", -",вопроса,"Разве мог Александр Мелявский, ступая на перро...","- Катенька иди, мы сами справимся. - [MASK] од...",вопрос
4,5,"Разве мог Александр Мелявский, ступая на перро...","Разве мог Александр Мелявский, ступая на перро...",<said aloud='True' type='direct'>- Катенька ид...,"- Катенька иди, мы сами справимся. - Сказала о...","поддержала её, вторая женщина. -Я подожду на у...","<speech_verb semantic=""speech"" emotion=""neutra...","сказал Саша, закрывая дверь кабинета, и обрадо...",сказал,"Разве мог Александр Мелявский, ступая на перро...","- Катенька иди, мы сами справимся. - [MASK] од...",сказать
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1039,(фарс-пародия к/ф «Иван Васильевич меняет проф...,(фарс-пародия к/ф «Иван Васильевич меняет проф...,<said aloud='True' type='direct'>- А закусываю...,- А закусываю только после двух подряд! -- гор...,- А закусываю только после двух подряд,"гордо <speech_verb semantic=""speech"" emotion=""...",гордо ответил Владимир и хряпнул второй стопарь.,ответил,(фарс-пародия к/ф «Иван Васильевич меняет проф...,- А закусываю только после двух подряд! -- гор...,ответить
996,1040,(фарс-пародия к/ф «Иван Васильевич меняет проф...,(фарс-пародия к/ф «Иван Васильевич меняет проф...,<said aloud='True' type='direct'>- Оставь меня...,"- Оставь меня старушка, я в астрале! Хочу -- р...","- Оставь меня старушка, я в астрале! Хочу -- р...","<speech_verb semantic=""speech"" emotion=""neutra...",сказал ей Владимир Владимирович и вернулся в к...,сказал,(фарс-пародия к/ф «Иван Васильевич меняет проф...,"- Оставь меня старушка, я в астрале! Хочу -- р...",сказать
997,1041,(фарс-пародия к/ф «Иван Васильевич меняет проф...,(фарс-пародия к/ф «Иван Васильевич меняет проф...,<said aloud='True' type='direct'>- Советские л...,- Советские люди не могут продать Родину и в И...,- Советские люди не могут продать Родину и в И...,"гордо <speech_verb semantic=""speech"" emotion=""...",гордо сказал Бунша и тут же осекся. На полу шк...,сказал,(фарс-пародия к/ф «Иван Васильевич меняет проф...,- Советские люди не могут продать Родину и в И...,сказать
998,1042,(фарс-пародия к/ф «Иван Васильевич меняет проф...,(фарс-пародия к/ф «Иван Васильевич меняет проф...,"<said aloud=

Initiate tokenizer

In [ ]:
# instantiate T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# check token ids
tokenizer.eos_token_id

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


1

In [ ]:
tokenizer.bos_token_id
tokenizer.unk_token_id

2

In [ ]:
tokenizer.pad_token_id

0

Create a new df for training

In [ ]:
df_class = df[['maske_ds', 'verb_lemma']]

In [ ]:
def tokenize_corpus(df, tokenizer, max_len):
    # token ID storage
    input_ids = []
    # attension mask storage
    attention_masks = []
    # max len -- 512 is max
    max_len = max_len
    # for every document:
    for doc in df:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            doc,  # document to encode.
                            add_special_tokens=True,  # add tokens relative to model
                            max_length=max_len,  # set max length
                            truncation=True,  # truncate longer messages
                            pad_to_max_length=True,  # add padding
                            return_attention_mask=True,  # create attn. masks
                            return_tensors='pt'  # return pytorch tensors
                       )

        # add the tokenized sentence to the list
        input_ids.append(encoded_dict['input_ids'])

        # and its attention mask (differentiates padding from non-padding)
        attention_masks.append(encoded_dict['attention_mask'])

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)


# create tokenized data
body_input_ids, body_attention_masks = tokenize_corpus(df_class['maske_ds'].values, tokenizer, 512)



# how long are tokenized targets
ls = []
for i in range(df_class.shape[0]):
    ls.append(len(tokenizer.tokenize(df_class.iloc[i]['verb_lemma'])))

temp_df = pd.DataFrame({'len_tokens': ls})
temp_df['len_tokens'].mean()  # 2.7

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


6.973

In [ ]:
temp_df['len_tokens'].median()

7.0

In [ ]:
temp_df['len_tokens'].max() 

11

In [ ]:
target_input_ids, target_attention_masks = tokenize_corpus(df_class['maske_ds'].values, tokenizer, 3)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
def prepare_dataset(body_tokens, body_masks, target_token, target_masks):
    # create tensor data sets
    tensor_df = TensorDataset(body_tokens, body_masks, target_token, target_masks)
    # 80% of df
    train_size = int(0.8 * len(df_class))
    # 20% of df
    val_size = len(df_class) - train_size
    # 50% of validation
    test_size = int(val_size - 0.5*val_size)
    # divide the dataset by randomly selecting samples
    train_dataset, val_dataset = random_split(tensor_df, [train_size, val_size])
    # divide validation by randomly selecting samples
    val_dataset, test_dataset = random_split(val_dataset, [test_size, len(val_dataset) - test_size])

    return train_dataset, val_dataset, test_dataset


# create tensor data sets
train_dataset, val_dataset, test_dataset = prepare_dataset(body_input_ids,
                                                           body_attention_masks,
                                                           target_input_ids,
                                                           target_attention_masks
                                                           )

In [ ]:
def train(model, dataloader, optimizer):

    # capture time
    total_t0 = time.time()

    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, epochs))
    print('Training...')

    # reset total loss for epoch
    train_total_loss = 0
    total_train_f1 = 0

    # put model into traning mode
    model.train()

    # for each batch of training data...
    for step, batch in enumerate(dataloader):

        # progress update every 40 batches.
        if step % 40 == 0 and not step == 0:

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(dataloader)))

        # Unpack this training batch from our dataloader:
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input tokens
        #   [1]: attention masks
        #   [2]: target tokens
        #   [3]: target attenion masks
        b_input_ids = batch[0].cuda()
        b_input_mask = batch[1].cuda()
        b_target_ids = batch[2].cuda()
        b_target_mask = batch[3].cuda()

        # clear previously calculated gradients
        optimizer.zero_grad()

        # runs the forward pass with autocasting.
        with autocast():
            # forward propagation (evaluate model on training batch)
            outputs = model(input_ids=b_input_ids,
                            attention_mask=b_input_mask,
                            labels=b_target_ids,
                            decoder_attention_mask=b_target_mask)

            loss, prediction_scores = outputs[:2]

            # sum the training loss over all batches for average loss at end
            # loss is a tensor containing a single value
            train_total_loss += loss.item()

        # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
        # Backward passes under autocast are not recommended.
        # Backward ops run in the same dtype autocast chose for corresponding forward ops.
        scaler.scale(loss).backward()

        # scaler.step() first unscales the gradients of the optimizer's assigned params.
        # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
        # otherwise, optimizer.step() is skipped.
        scaler.step(optimizer)

        # Updates the scale for next iteration.
        scaler.update()

        # update the learning rate
        scheduler.step()

    # calculate the average loss over all of the batches
    avg_train_loss = train_total_loss / len(dataloader)

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'Train Loss': avg_train_loss
        }
    )

    # training time end
    training_time = format_time(time.time() - total_t0)

    # print result summaries
    print("")
    print("summary results")
    print("epoch | trn loss | trn time ")
    print(f"{epoch+1:5d} | {avg_train_loss:.5f} | {training_time:}")

    return training_stats

In [ ]:
def validating(model, dataloader):

    # capture validation time
    total_t0 = time.time()

    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")

    # put the model in evaluation mode
    model.eval()

    # track variables
    total_valid_loss = 0

    # evaluate data for one epoch
    for batch in dataloader:

        # Unpack this training batch from our dataloader:
        # `batch` contains three pytorch tensors:
        #   [0]: input tokens
        #   [1]: attention masks
        #   [2]: target tokens
        #   [3]: target attenion masks
        b_input_ids = batch[0].cuda()
        b_input_mask = batch[1].cuda()
        b_target_ids = batch[2].cuda()
        b_target_mask = batch[3].cuda()

        # tell pytorch not to bother calculating gradients
        # as its only necessary for training
        with torch.no_grad():

            # forward propagation (evaluate model on training batch)
            outputs = model(input_ids=b_input_ids,
                            attention_mask=b_input_mask,
                            labels=b_target_ids,
                            decoder_attention_mask=b_target_mask)

            loss, prediction_scores = outputs[:2]

            # sum the training loss over all batches for average loss at end
            # loss is a tensor containing a single value
            total_valid_loss += loss.item()

    # calculate the average loss over all of the batches.
    global avg_val_loss
    avg_val_loss = total_valid_loss / len(dataloader)

    # Record all statistics from this epoch.
    valid_stats.append(
        {
            'Val Loss': avg_val_loss,
            'Val PPL.': np.exp(avg_val_loss)
        }
    )

    # capture end validation time
    training_time = format_time(time.time() - total_t0)

    # print result summaries
    print("")
    print("summary results")
    print("epoch | val loss | val ppl | val time")
    print(f"{epoch+1:5d} | {avg_val_loss:.5f} | {np.exp(avg_val_loss):.3f} | {training_time:}")

    return valid_stats


def testing(model, dataloader):

    print("")
    print("Running Testing...")

    # measure training time
    t0 = time.time()

    # put the model in evaluation mode
    model.eval()

    # track variables
    total_test_loss = 0
    total_test_acc = 0
    total_test_f1 = 0
    predictions = []
    actuals = []

    # evaluate data for one epoch
    for step, batch in enumerate(dataloader):
        # progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(dataloader), elapsed))

        # Unpack this training batch from our dataloader:
        # `batch` contains three pytorch tensors:
        #   [0]: input tokens
        #   [1]: attention masks
        #   [2]: target tokens
        #   [3]: target attenion masks
        b_input_ids = batch[0].cuda()
        b_input_mask = batch[1].cuda()
        b_target_ids = batch[2].cuda()
        b_target_mask = batch[3].cuda()

        # tell pytorch not to bother calculating gradients
        # as its only necessary for training
        with torch.no_grad():

            # forward propagation (evaluate model on training batch)
            outputs = model(input_ids=b_input_ids,
                            attention_mask=b_input_mask,
                            labels=b_target_ids,
                            decoder_attention_mask=b_target_mask)

            loss, prediction_scores = outputs[:2]

            total_test_loss += loss.item()

            generated_ids = model.generate(
                    input_ids=b_input_ids,
                    attention_mask=b_input_mask,
                    max_length=3
                    )

            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in b_target_ids]

            total_test_acc += accuracy_score(target, preds)
            total_test_f1 += f1_score(preds, target,
                                       average='weighted',
                                       labels=np.unique(preds))
            predictions.extend(preds)
            actuals.extend(target)

    # calculate the average loss over all of the batches.
    avg_test_loss = total_test_loss / len(dataloader)

    avg_test_acc = total_test_acc / len(test_dataloader)

    avg_test_f1 = total_test_f1 / len(test_dataloader)

    # Record all statistics from this epoch.
    test_stats.append(
        {
            'Test Loss': avg_test_loss,
            'Test PPL.': np.exp(avg_test_loss),
            'Test Acc.': avg_test_acc,
            'Test F1': avg_test_f1
        }
    )
    global df2
    temp_data = pd.DataFrame({'predicted': predictions, 'actual': actuals})
    df2 = df2.append(temp_data)

    return test_stats
    

# time function
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# instantiate model T5 transformer with a language modeling head on top
model = T5ForConditionalGeneration.from_pretrained('t5-small').cuda()  # to GPU

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
train_dataloader = DataLoader(train_dataset,
                              batch_size=24,
                              # sampler=train_sampler,
                              shuffle=False)

valid_dataloader = DataLoader(val_dataset,
                              batch_size=24,
                              shuffle=True)

test_dataloader = DataLoader(test_dataset,
                              batch_size=24,
                              shuffle=True)


# Adam w/ Weight Decay Fix
# set to optimizer_grouped_parameters or model.parameters()
optimizer = AdamW(model.parameters(),
                  lr = 3e-5
                  )

# epochs
epochs = 6

# lr scheduler
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

# create gradient scaler for mixed precision
scaler = GradScaler()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
# create training result storage
training_stats = []
valid_stats = []
best_valid_loss = float('inf')

# for each epoch
for epoch in range(epochs):
    # train
    train(model, train_dataloader, optimizer)
    # validate
    validating(model, valid_dataloader)
    # check validation loss
    if valid_stats[epoch]['Val Loss'] < best_valid_loss:
        best_valid_loss = valid_stats[epoch]['Val Loss']
        # save best model for use later
        torch.save(model.state_dict(), 't5-classification.pt')  # torch save
        model_to_save = model.module if hasattr(model, 'module') else model
        model_to_save.save_pretrained('./model_save/t5-classification/')  # transformers save
        tokenizer.save_pretrained('./model_save/t5-classification/')  # transformers save


======== Epoch 1 / 6 ========
Training...


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)



summary results
epoch | trn loss | trn time 
    1 | 3.04583 | 0:00:15

Running Validation...

summary results
epoch | val loss | val ppl | val time
    1 | 0.86599 | 2.377 | 0:00:01

======== Epoch 2 / 6 ========
Training...

summary results
epoch | trn loss | trn time 
    2 | 0.79236 | 0:00:12

Running Validation...

summary results
epoch | val loss | val ppl | val time
    2 | 0.20873 | 1.232 | 0:00:01

======== Epoch 3 / 6 ========
Training...

summary results
epoch | trn loss | trn time 
    3 | 0.29711 | 0:00:12

Running Validation...

summary results
epoch | val loss | val ppl | val time
    3 | 0.07883 | 1.082 | 0:00:01

======== Epoch 4 / 6 ========
Training...

summary results
epoch | trn loss | trn time 
    4 | 0.18485 | 0:00:12

Running Validation...

summary results
epoch | val loss | val ppl | val time
    4 | 0.05728 | 1.059 | 0:00:01

======== Epoch 5 / 6 ========
Training...

summary results
epoch | trn loss | trn time 
    5 | 0.14208 | 0:00:12

Running Validation.

In [ ]:
# organize results
pd.set_option('precision', 3)
df_train_stats = pd.DataFrame(data=training_stats)
df_valid_stats = pd.DataFrame(data=valid_stats)
df_stats = pd.concat([df_train_stats, df_valid_stats], axis=1)
df_stats.insert(0, 'Epoch', range(1, len(df_stats)+1))
df_stats = df_stats.set_index('Epoch')
df_stats

,Train Loss,Val Loss,Val PPL.
Epoch,,,
1,3.046,0.866,2.377
2,0.792,0.209,1.232
3,0.297,0.079,1.082
4,0.185,0.057,1.059
5,0.142,0.045,1.046
6,0.132,0.053,1.054


In [ ]:
# test the model
df2 = pd.DataFrame({'predicted': [], 'actual': []})
test_stats = []
model.load_state_dict(torch.load('t5-classification.pt'))

<All keys matched successfully>

In [ ]:
testing(model, test_dataloader)


Running Testing...


[{'Test Loss': 0.04188517183065414,
  'Test PPL.': 1.0427747319622311,
  'Test Acc.': 0.9666666666666668,
  'Test F1': 0.9653385503385504}]

In [ ]:
df_test_stats = pd.DataFrame(data=test_stats)
print(df_test_stats)

   Test Loss  Test PPL.  Test Acc.  Test F1
0      0.042      1.043      0.967    0.965
